In [31]:
"""
If you want to use new map results, then go to tournament_reduce.py, and replace the extract() method with the following code. 
This will save the map results for use in this notebook. Once those are saved, switch the original code back. 
"""

async def extract(
    self, results: list[LookupOutput], district: District, term: str
):
    # We first map extraction across all pages.
    results = []
    empty_results = []
    map_pickle = []
    async for r in super().extract(pages, district, term):
        map_pickle.append(r)
        if (r.output is not None) and r.output.extracted_text:
            results.append(r)
        else:
            empty_results.append(r)
            
    # Load existing data if the file already exists
    try:
        with open("./ryurtyn/map_results_11_14_0.dat", "rb") as f:
            existing_data = pickle.load(f)
    except FileNotFoundError:
        existing_data = {}  # If the file doesn't exist, initialize an empty dictionary

    try:
        with open("./ryurtyn/districts_11_14_0.dat", "rb") as f:
            existing_data_district = pickle.load(f)
    except FileNotFoundError:
        existing_data_district = {} 
    # Assuming district.short_name and results are defined somewhere in your code
    new_data = {district.short_name: map_pickle}
    new_data_district = {district.short_name: district}

    # Merge the existing data with the new data
    existing_data.update(new_data)
    existing_data_district.update(new_data_district)

    # Write the updated data back to the file
    with open("./ryurtyn/map_results_11_14_0.dat", "wb") as f:
        pickle.dump(existing_data, f)

    with open("./ryurtyn/districts_11_14_0.dat", "wb") as f:
        pickle.dump(existing_data_district, f)

    # Ensure that we yield one empty result to handle case when the expected output is None
    if len(empty_results) != 0:
        yield empty_results[0]

NameError: name 'LookupOutput' is not defined

In [42]:
import pickle
from zoning.term_extraction.extract.tournament_reduce import TournamentReduceExtractor, tournament_reduce
import asyncio
import pandas as pd

In [38]:
# load the map outputs
with open("map_results_11_14_0.dat", "rb") as f:
    map_outputs = pickle.load(f)

# load the dictionary of district objects
with open("districts_11_14_0.dat", "rb") as f:
    districts = pickle.load(f)

In [39]:
# get districts 
districts_without_ground_truth = ["TD", "IP", "VDO", "IND-1", "GB", "MXN", "MX2", "AZ"]
districts_with_ground_truth = [d for d in districts.keys() if d not in districts_without_ground_truth]
len(districts_without_ground_truth), len(districts_with_ground_truth)

(8, 20)

In [40]:
# Define an asynchronous function that executes the coroutine object
async def execute_tournament_test(inputs, term, district):
    # Your coroutine object (replace this with your actual coroutine)
    async def my_coroutine():
        return await tournament_reduce(inputs, term, district, 1)
    # Execute the coroutine object using await
    return await my_coroutine()

In [36]:
"""
When you're ready to run the experiment, replace the extract method of tournament with this code.
This ensure that you don't map extract, and just run tournamnet on your pair of elements
"""
async def extract(
    self, pages: list[LookupOutput], district: District, term: str
):
    results = pages
    for result in await tournament_reduce(results, term, district, self.k):
        yield result

NameError: name 'LookupOutput' is not defined

Comparing Results Where GT doesn't exist

In [41]:
total_length = 0
total_selected_index = 0
total_selected_none = 0


for d in districts_without_ground_truth:
    inputs = map_outputs[d]
    district = districts[d]
    term = "min_lot_size"
    print("District: ", district)

    selected_index = 0
    selected_none = 0
    total_length += len(inputs)
    for a in range(len(inputs)):
        i = inputs[a]
        if i.output and i.output.extracted_text:
            for b in range(a + 1, len(inputs)):
                j = inputs[b]
                if j.output and j.output.extracted_text:
                    res = await execute_tournament_test([i, j], term, district)
                    if res == -1:
                        selected_none += 1
                        total_selected_none += 1
                        
                    elif res == 1 or res == 0:
                        selected_index += 1
                        total_selected_index += 1
                    

    print("Selected None: ", selected_none)
    print("Selected Index: ", selected_index)
    print("Accuracy: ", selected_none / len(inputs))

print("Final Selected None: ", total_selected_none)
print("Final Accuracy: ", total_selected_none / total_length)

District:  full_name='Technology Development' short_name='TD'
Selected None:  0
Selected Index:  3
Accuracy:  0.0
District:  full_name='Industrial Park' short_name='IP'


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:37: UserWarning: Phrase CELL (1, 1): 
Minimum lot area was found more than once in the document.
  warnings.warn(f"Phrase {phrase} was found more than once in the document.")


Selected None:  0
Selected Index:  28
Accuracy:  0.0
District:  full_name='Village Overlay' short_name='VDO'


CancelledError: 

Comparing Results Where GT exists

In [43]:
gt = pd.read_csv("../data/ground_truth.csv")
gt_first_30 = gt.head(30)

In [67]:
gt_first_30[gt_first_30["district_abb"] == "AA"]["min_lot_size_gt"][1]

'30000'

In [76]:
from zoning.term_extraction.eval_results import clean_string_units

In [77]:
clean_string_units(map_outputs["AA"][0].output.answer)[0]

[30000.0]

In [80]:
true_val = gt_first_30[gt_first_30["district_abb"] == "AL"]["min_lot_size_gt"][1]

KeyError: 1

In [79]:
total_length = 0
total_selected_correct = 0
total_selected_incorrect = 0


for d in districts_with_ground_truth:
    inputs = map_outputs[d]
    district = districts[d]
    print(d)
    try:
        true_val = gt_first_30[gt_first_30["district_abb"] == d]["min_lot_size_gt"][1]
    except KeyError:
        print("skipping ", d)
        continue

    term = "min_lot_size"
    print("District: ", district)

    selected_correct = 0
    selected_incorrect = 0
    total_length += len(inputs)
    # find which one is true answer
    true_index = None
    for a in range(len(inputs)):
        i = inputs[a]
        if i.output and i.output.answer:
            print(true_val, i.output.answer)
            i_answer_clean = clean_string_units(i.output.answer)[0]
            if i_answer_clean == true_val:
                true_index = a
                continue
    # if you found right answer, that becomes the 0th index you pass in to tournament
    if true_index:
        for a in range(len(inputs)):
            if a != true_index:
                i = inputs[a]
                if i.output and i.output.extracted_text:
                    res = await execute_tournament_test([i, j], term, district)
                    if res == 0:
                        selected_correct += 1
                        total_selected_correct += 1 
                    if res == 1:
                        selected_incorrect += 1
                        total_selected_incorrect += 1
    else:
        print("true answer not found in answers?? check this ")

    print("Selected Correct: ", selected_correct)
    print("Selected Incorrect: ", selected_incorrect)
    print("Accuracy: ", selected_correct / len(inputs))

print("Final Selected None: ", total_selected_correct)
print("Final Accuracy: ", total_selected_correct / total_length)

AL


KeyError: 1

Original Zero-Shot Prompt: 0 nones selected

Updated Zero-Shot Prompt: 0 nones selected

In [ ]:

districts_with_ground_truth = ["B-2"]
# for d in districts_without_ground_truth:
for d in districts_with_ground_truth:
    inputs = map_outputs[d]
    district = districts[d]
    term = "min_lot_size"
    # print("map outputs is ", inputs)
    print("District: ", district)


    i = 0
    while not i.output or not i.output.extracted_text:
        i += 1
    for j in range(i + 1, len(inputs)):
        if i.output and j.output.extracted_text:
            if j.output and j.output.extracted_text:
                        res = await execute_tournament_test([i, j], term, district)
                        if res == -1:
                            reported_none += 1
                            total_reported_none += 1
                        print("i0: ", i.output.extracted_text[0], "\ni1: ",  j.output.extracted_text[0], "\n index: ", res)
        